In [1]:
import pandas as pd
import numpy as np
import math
import json

% matplotlib inline

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

UsageError: Line magic function `%` not found.


In [2]:
def id_mapper(df , column):
    '''
    Map a column in a DataFrame and create a dict to change its value to a sequence ('1','2','3'...) as str for easier use.
    
    INPUT:
    df - (DataFrame) 
    column - (str) name of the column to create dictionary
    OUTPUT:
    coded_dict - (dict) A dictionary with the given column values as key and the 'new' encoded sequence as value
    
    '''  
    coded_dict = dict()
    cter = 1
    
    for x in df[column]:
        if x not in coded_dict:
            coded_dict[x] = str(cter)
            cter+=1
            
    return coded_dict

In [3]:
def offer_mapper(df=portfolio):
    '''
    Check if one offer has been sent more than one time and then merge rows from the same offer into one row
    
    IMPUT: df - (DataFrame) - portfolio as default dataframe 
    OUTPUT: df - (DataFrame) - relation between offer id and type of offer
    
    '''
    
    # get sequence to name type of offers
    coded_dict = id_mapper(df, 'offer_type')
    
    coded_df = df.replace({"offer_type":coded_dict})
    offer_type = coded_df[['id' , 'offer_type']]
    duration = coded_df[['id' , 'duration']]
    
    return offer_type

In [4]:
def arrange_events(df=transcript):
    '''
    Rearange the dataframe transcript, by reading the dictionary in the column 'value' 
    and merging each offer with the transaction value
    
    IMPUT: df (DataFrame) - default Dataframe is transcript
    OUTPUT: df (DataFrame) - modified dataFrame
    
    '''
    
    # read dictionary from 'value' feature and create columns 
    df = pd.concat([df.drop(['value'], axis=1), df['value'].apply(pd.Series)], axis=1)
    
    # merge offer id and offer_id columns
    df['offer id'] = df['offer id'].combine_first(df['offer_id'])
    df = df.drop(columns = ['offer_id'])
    
    # split into two dataFrames to merge rows with transaction and offer recived in the same time
    df1 = df[df['event'] != 'transaction'][['event' , 'person' , 'time' , 'offer id' , 'reward']]
    df2 = df[df['event'] == 'transaction'][['person' , 'time' , 'amount']]
    
    # merge the two dataFrames on time
    df = pd.merge(df1, df2, how='outer', on=['person', 'time'])
    
    # create columns of type of event with the value of time
    df = pd.concat([df, df.pivot_table(values='time', index=df.index, columns='event', aggfunc='first')], axis=1, sort=False)
    
    # fill NaN values in the offer id feature as 'no offer' to keep track of the transactions without an offer
    df['offer id'] = df['offer id'].fillna(value = 'no offer')
    df = df.rename(columns={'person':'user id'})
    
    # drop event and time columns
    df = df.drop(columns=['event' , 'time'])
    
    return df

In [5]:
def offer_merge(df=transcript):
    
    '''
    For a given user, checks and select rows of an offers with different events and then merge them into one row.
    
    IMPUT: df - (DataFrame) - transcription as default dataframe.
    OUTPUT: df - (DataFrame) - rearange user data where each offer is in one row. 
    '''
    
    offers_received_lst = df['offer id'].unique().tolist()
    total_offers_received = df['offer received'].count()
    temp_df = df.head(0)
    user_id = df['user id'].unique()[0]
    
    
    for offer in offers_received_lst:
        
        #create data frame of an offer
        offer_df = df[df['offer id'] == offer].copy()
        
        # check if the same offer has been receved more than one time if so, create flags to treat each offer independently.
        if offer_df['offer received'].count() > 1:
            cter = 0
            flag = []
            
            #create list to flag each offer            
            for index, row in offer_df.iterrows():
                if not np.isnan(row['offer received']):
                    cter+=1
                    flag.append(cter)
                    
                else:
                    flag.append(cter)
                    
            offer_df['flag'] = flag 
            offer_df = offer_df.groupby(['flag' , 'offer id']).mean().reset_index().drop(columns='flag')

        else:
            offer_df = offer_df.groupby('offer id').mean().reset_index()
            
        temp_df = temp_df.append(offer_df , sort=False)
    
    df = temp_df
    
    return df , user_id

In [25]:
def check_completed_offers(df , user_id):
    '''
    For a given user, checks and drop transactions that were not influenced by an offer
    
    IMPUT: df - (DataFrame)
    OUTPUT: df - (DataFrame) - rearange data 
    '''
  
    # fill NaN values with 0 for offers that were not completed
    df[['reward' , 'amount']] = df[['reward' , 'amount']].fillna(value = 0)
    
    # add column with the type of offer   
    df = pd.merge(df, offer_type_mapper, how='left', left_on=['offer id'] , right_on=['id'])
    
    # fill with offer type 4, for transactions that are not related with an offer
    df['offer_type'] = df['offer_type'].fillna(value = str(4))
    df['user id'] = df['user id'].fillna(value = user_id)
    
    # check if an offer was completed before it was viewed or if it was not viewed, if so, drop it (the offer did not influenciate the transaction)
    for row in range(len(df)):
        if df.loc[row]['offer viewed'] > df.loc[row]['offer completed']:
            df = df.drop([row])
        elif np.isnan(df.loc[row]['offer viewed']) and not np.isnan(df.loc[row]['offer completed']):
            df = df.drop([row])
        else:
            pass
        
    return df

In [7]:
def get_events(df):
    '''
    for each user rearange transactions influenced by an offer
    and for each type of offer get:
    transaction amount, number of offers recived, number of offers viewed and number of offers completed
    note: it takes some time to process
    
    IMPUT: df - (dtaFrame)
    OUTPUT:
        amount_lst (lst) - list of dictionaries that contains amount spend and type of offer for each user
        offers_lst (lst) - list of dictionaries that contains number of offers recived for each type
        offers_view_lst (lst) - list of dictionaries that contains number of offers viewed for each type
        offers_completed_lst (lst) - list of dictionaries that contains number of offers completed for each type
    
    '''
    
    user_id_lst = profile['id'].tolist()
    amount_lst = []
    offers_lst = []
    offers_view_lst = []
    offers_completed_lst = []
    
    for user in user_id_lst:
        
        user_arrange_transcript = df[df['user id'] == user]
        user_events, user_id = offer_merge(user_arrange_transcript)
        user_events = check_completed_offers (user_events , user_id)

        amount = {'user id' : user}
        offers = {'user id' : user}
        offers_view = {'user id' : user}
        offers_completed = {'user id' : user}

        amount.update(user_events.groupby('offer_type').mean()['amount'].to_dict())
        offers.update(user_events.groupby('offer_type').count()['offer id'].to_dict())
        offers_view.update(user_events.groupby('offer_type').count()['offer viewed'].to_dict())
        offers_completed.update(user_events.groupby('offer_type').count()['offer completed'].to_dict())

        amount_lst.append(amount)
        offers_lst.append(offers)
        offers_view_lst.append(offers_view)
        offers_completed_lst.append(offers_completed)
        
    return amount_lst , offers_lst , offers_view_lst , offers_completed_lst

In [8]:
# split gender into dummies columns
profile_mod = pd.concat([profile , pd.get_dummies(profile['gender'])],axis=1)
profile_mod.drop(['gender' , 'became_member_on'],axis=1, inplace=True)

profile_mod.head()

,age,id,income,F,M,O
0,118,68be06ca386d4c31939f3a4f0e3dd783,NaN,0,0,0
1,55,0610b486422d4921ae7d2bf64640c50b,112000.0,1,0,0
2,118,38fe809add3b4fcf9315a9694bb96ff5,NaN,0,0,0
3,75,78afa995795e4d85b5d9ceeca43f5fef,100000.0,1,0,0
4,118,a03223e636434f42ac4c3df47e8bac43,NaN,0,0,0


# main

In [9]:
#map offer type 
offer_type_mapper = offer_mapper(portfolio)

# arrange transcript df
arrange_transcript = arrange_events(transcript)
arrange_transcript.head()

,user id,offer id,reward,amount,offer completed,offer received,offer viewed
0,78afa995795e4d85b5d9ceeca43f5fef,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,NaN,0.0,NaN
1,a03223e636434f42ac4c3df47e8bac43,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,NaN,0.0,NaN
2,e2127556f4f64592b11af22de27a7932,2906b810c7d4411798c6938adc9daaa5,NaN,NaN,NaN,0.0,NaN
3,8ec6ce2a7e7949b1bf142def7d0e0586,fafdcd668e3743c1bb461111dcafc2a4,NaN,NaN,NaN,0.0,NaN
4,68617ca6246f4fbc85e91a2a49552598,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,NaN,NaN,0.0,NaN


In [10]:
#note: this will take some time to execute you can grab a coffee if you like
amount_lst , offers_lst , offers_view_lst , offers_completed_lst = get_events(arrange_transcript)

In [38]:
# Amount data
amount_type = pd.DataFrame(amount_lst)
amount_type.rename(columns={'1': 'type 1', '2': 'type 2', '3': 'type 3' , '4': 'no offer'} , inplace = True)
amount_type.head()

,type 1,type 2,type 3,no offer,user id
0,NaN,NaN,0.848,2.285000,68be06ca386d4c31939f3a4f0e3dd783
1,NaN,0.0,NaN,26.895000,0610b486422d4921ae7d2bf64640c50b
2,0.000,0.0,NaN,2.383333,38fe809add3b4fcf9315a9694bb96ff5
3,20.805,0.0,NaN,23.532000,78afa995795e4d85b5d9ceeca43f5fef
4,NaN,0.0,0.000,1.550000,a03223e636434f42ac4c3df47e8bac43


In [39]:
def df_from_lst (lst):
    '''
    create dataframe from a list of dictionaries
    IMPUT: lst (list)
    OUTPUT: DF (dataFrame)
     '''
    
    df = pd.DataFrame(lst).drop(columns='4')
    df.fillna(value = 0 , inplace = True)
    
    return df    

In [40]:
user_offers = df_from_lst (offers_lst)
offers_viewed = df_from_lst (offers_view_lst)
offers_completed = df_from_lst (offers_completed_lst)

user_offers.rename(columns={'1': 'offers type 1', '2': 'offers type 2', '3': 'offers type 3'} , inplace = True)
offers_viewed.rename(columns={'1': 'viewed type 1', '2': 'viewed type 2', '3': 'viewed type 3'} , inplace = True)
offers_completed.rename(columns={'1': 'completed type 1', '2': 'completed type 2', '3': 'completed type 3'} , inplace = True)

In [41]:
#profile_amount_type = pd.merge(profile_mod, amount_type, how='inner' , left_on="id" , right_on="user id")
#profile_amount_type.drop(['id'],axis=1, inplace=True)

amount_offer = pd.merge(amount_type, user_offers, how='inner' , on="user id")
amount_offer = pd.merge(amount_offer, offers_viewed, how='inner' , on="user id")
amount_offer = pd.merge(amount_offer, offers_completed, how='inner' , on="user id")
amount_offer.head()

,type 1,type 2,type 3,no offer,user id,offers type 1,offers type 2,offers type 3,viewed type 1,viewed type 2,viewed type 3,completed type 1,completed type 2,completed type 3
0,NaN,NaN,0.848,2.285000,68be06ca386d4c31939f3a4f0e3dd783,0.0,0.0,5.0,0.0,0.0,5.0,0.0,0.0,2.0
1,NaN,0.0,NaN,26.895000,0610b486422d4921ae7d2bf64640c50b,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000,0.0,NaN,2.383333,38fe809add3b4fcf9315a9694bb96ff5,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
3,20.805,0.0,NaN,23.532000,78afa995795e4d85b5d9ceeca43f5fef,2.0,1.0,0.0,2.0,1.0,0.0,2.0,0.0,0.0
4,NaN,0.0,0.000,1.550000,a03223e636434f42ac4c3df47e8bac43,0.0,2.0,3.0,0.0,1.0,2.0,0.0,0.0,0.0


In [42]:
#split into 3 datasets
amount_type_1 = amount_offer[['user id' , 'type 1' , 'offers type 1' , 'viewed type 1' , 'completed type 1']].copy()
amount_type_2 = amount_offer[['user id' , 'type 2' , 'offers type 2' , 'viewed type 2' , 'completed type 2']].copy()
amount_type_3 = amount_offer[['user id' , 'type 3' , 'offers type 3' , 'viewed type 3' , 'completed type 3']].copy()

In [43]:
amount_type_1 = pd.merge(profile_mod, amount_type_1, how='inner' , left_on="id" , right_on="user id")
amount_type_1.drop(['id'],axis=1, inplace=True)

amount_type_2 = pd.merge(profile_mod, amount_type_2, how='inner' , left_on="id" , right_on="user id")
amount_type_2.drop(['id'],axis=1, inplace=True)

amount_type_3 = pd.merge(profile_mod, amount_type_3, how='inner' , left_on="id" , right_on="user id")
amount_type_3.drop(['id'],axis=1, inplace=True)

In [44]:
# clear NaN for each data Set, users that diddent recive that type of offer
amount_type_1.dropna(axis=0 , inplace=True)
amount_type_1.drop(amount_type_1[amount_type_1['viewed type 1'] == 0].index , inplace=True)

amount_type_2.dropna(axis=0 , inplace=True)
amount_type_2.drop(amount_type_2[amount_type_2['viewed type 2'] == 0].index , inplace=True)

amount_type_3.dropna(axis=0 , inplace=True)
amount_type_3.drop(amount_type_3[amount_type_3['viewed type 3'] == 0].index , inplace=True)

In [48]:
# save dataFrames to CSV files
amount_type_1.to_csv('data/amount_type_1.csv' , index=False)
amount_type_2.to_csv('data/amount_type_2.csv' , index=False)
amount_type_3.to_csv('data/amount_type_3.csv' , index=False)